In [1]:
# encoding=utf8  
import numpy as np
import tensorflow as tf
import config
def unpickle(file):
  import _pickle as cPickle
  fo = open(file, 'rb')
  dict = cPickle.load(fo,encoding='latin1')
  fo.close()
  if 'data' in dict:
    dict['data'] = dict['data'].reshape((-1, 3, 32, 32)).swapaxes(1, 3).swapaxes(1, 2).reshape(-1, 32*32*3) / 256.

  return dict

In [2]:
def load_data_one(f):
  batch = unpickle(f)
  data = batch['data']
  labels = batch['labels']
  print ("Loading %s: %d" % (f, len(data)))
  return data, labels


In [3]:

def load_data(files, data_dir, label_count):
  data, labels = load_data_one(data_dir + '/' + files[0])
  for f in files[1:]:
    data_n, labels_n = load_data_one(data_dir + '/' + f)
    data = np.append(data, data_n, axis=0)
    labels = np.append(labels, labels_n, axis=0)
  labels = np.array([ [ float(i == label) for i in range(label_count) ] for label in labels ])
  return data, labels

In [4]:
def run_in_batch_avg(session, tensors, batch_placeholders, feed_dict={}, batch_size=200):                              
  res = [ 0 ] * len(tensors)                                                                                           
  batch_tensors = [ (placeholder, feed_dict[ placeholder ]) for placeholder in batch_placeholders ]                    
  total_size = len(batch_tensors[0][1])                                                                                
  batch_count = (total_size + batch_size - 1) / batch_size                                                             
  for batch_idx in range(batch_count):                                                                                
    current_batch_size = None                                                                                          
    for (placeholder, tensor) in batch_tensors:                                                                        
      batch_tensor = tensor[ batch_idx*batch_size : (batch_idx+1)*batch_size ]                                         
      current_batch_size = len(batch_tensor)                                                                           
      feed_dict[placeholder] = tensor[ batch_idx*batch_size : (batch_idx+1)*batch_size ]                               
    tmp = session.run(tensors, feed_dict=feed_dict)                                                                    
    res = [ r + t * current_batch_size for (r, t) in zip(res, tmp) ]                                                   
  return [ r / float(total_size) for r in res ]


In [5]:
def add_dyprune(weights):
    import config
    crate = config.crate[weights.name[:-2]]
    print(weights.name[:-2])
    prune_mask = tf.Variable(tf.ones_like(weights),name=weights.name[:-2]+'mask', trainable=False)

    mean = tf.divide(tf.reduce_sum(tf.multiply(tf.abs(weights),prune_mask)),tf.reduce_sum(prune_mask))
    
    var = tf.multiply(weights,prune_mask)
    var = tf.square(var)
    mean_q = tf.square(mean)*tf.reduce_sum(prune_mask)
    var = tf.reduce_sum(var) - mean_q
    
    var = tf.divide(var,tf.reduce_sum(prune_mask))
    var = tf.sqrt(var)
    
    t1_lower = (mean+var*crate)*0.45

    indicator_lower1 = tf.greater_equal(tf.abs(weights), tf.ones_like(weights) * t1_lower)
    
    t1_upper = (mean+var*crate)*0.55

    
    
    
    
    
    indicator_upper1 = tf.greater_equal(tf.abs(weights), tf.ones_like(weights) * t1_upper)
    indicator_matrix1 = tf.greater_equal(prune_mask, tf.zeros_like(weights))
    indicator_matrix1 = tf.logical_and(indicator_matrix1,indicator_lower1)
    indicator_matrix1 = tf.logical_or(indicator_matrix1,indicator_upper1)
    indicator_matrix1 = tf.to_float(indicator_matrix1)
    update = prune_mask.assign(indicator_matrix1)
    print(update.name)

    prune_fc = tf.multiply(weights, prune_mask)
    return prune_fc


def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)   
  w = tf.Variable(initial)
  if w.name[:-2] in config.all_para: w = add_dyprune(w)
  return w


def bias_variable(shape):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial)

def conv2d(input, in_features, out_features, kernel_size, with_bias=False):
  W = weight_variable([ kernel_size, kernel_size, in_features, out_features ])
  conv = tf.nn.conv2d(input, W, [ 1, 1, 1, 1 ], padding='SAME')
  if with_bias:
    return conv + bias_variable([ out_features ])
  return conv

def batch_activ_conv(current, in_features, out_features, kernel_size, is_training, keep_prob):
  current = tf.contrib.layers.batch_norm(current, scale=True, is_training=is_training, updates_collections=None)
  current = tf.nn.relu(current)
  current = conv2d(current, in_features, out_features, kernel_size)
  current = tf.nn.dropout(current, keep_prob)
  return current

def block(input, layers, in_features, growth, is_training, keep_prob):
  current = input
  features = in_features
  for idx in range(layers):
    tmp = batch_activ_conv(current, features, growth, 3, is_training, keep_prob)
    current = tf.concat((current, tmp),3)
    features += growth
  return current, features

def avg_pool(input, s):
  return tf.nn.avg_pool(input, [ 1, s, s, 1 ], [1, s, s, 1 ], 'VALID')



In [6]:
data_dir = './data'
image_size = 32
image_dim = image_size * image_size * 3
# meta = unpickle(data_dir + '/batches.meta')
# label_names = meta['label_names']
# label_count = len(label_names)
label_count = 10
# train_files = [ 'data_batch_%d' % d for d in range(1, 6) ]
# train_data, train_labels = load_data(train_files, data_dir, label_count)
# pi = np.random.permutation(len(train_data))
# train_data, train_labels = train_data[pi], train_labels[pi]
# test_data, test_labels = load_data([ 'test_batch' ], data_dir, label_count)
# print ("Train:", np.shape(train_data), np.shape(train_labels))
# print ("Test:", np.shape(test_data), np.shape(test_labels))
# data = { 'train_data': train_data,
#   'train_labels': train_labels,
#   'test_data': test_data,
#   'test_labels': test_labels }
depth = 40

In [7]:

weight_decay = 1e-4
layers = int((depth - 4) / 3)


xs = tf.placeholder("float", shape=[None, image_dim])
ys = tf.placeholder("float", shape=[None, label_count])
lr = tf.placeholder("float", shape=[])
keep_prob = tf.placeholder(tf.float32)
is_training = tf.placeholder("bool", shape=[])


current = tf.reshape(xs, [ -1, 32, 32, 3 ])
current = conv2d(current, 3, 16, 3)

current, features = block(current, layers, 16, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)

current = tf.contrib.layers.batch_norm(current, scale=True, is_training=is_training, updates_collections=None)
current = tf.nn.relu(current)
current = avg_pool(current, 8)
final_dim = features
current = tf.reshape(current, [ -1, final_dim ])
Wfc = weight_variable([ final_dim, label_count ])
bfc = bias_variable([ label_count ])
ys_ = tf.nn.softmax( tf.matmul(current, Wfc) + bfc )

cross_entropy = -tf.reduce_mean(ys * tf.log(ys_ + 1e-12))
l2 = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
train_step = tf.train.MomentumOptimizer(lr, 0.9, use_nesterov=True).minimize(cross_entropy + l2 * weight_decay)
correct_prediction = tf.equal(tf.argmax(ys_, 1), tf.argmax(ys, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    


Variable
Assign:0
Variable_1
Assign_1:0
Variable_2
Assign_2:0
Variable_3
Assign_3:0
Variable_4
Assign_4:0
Variable_5
Assign_5:0
Variable_6
Assign_6:0
Variable_7
Assign_7:0
Variable_8
Assign_8:0
Variable_9
Assign_9:0
Variable_10
Assign_10:0
Variable_11
Assign_11:0
Variable_12
Assign_12:0
Variable_13
Assign_13:0
Variable_14
Assign_14:0
Variable_15
Assign_15:0
Variable_16
Assign_16:0
Variable_17
Assign_17:0
Variable_18
Assign_18:0
Variable_19
Assign_19:0
Variable_20
Assign_20:0
Variable_21
Assign_21:0
Variable_22
Assign_22:0
Variable_23
Assign_23:0
Variable_24
Assign_24:0
Variable_25
Assign_25:0
Variable_26
Assign_26:0
Variable_27
Assign_27:0
Variable_28
Assign_28:0
Variable_29
Assign_29:0
Variable_30
Assign_30:0
Variable_31
Assign_31:0
Variable_32
Assign_32:0
Variable_33
Assign_33:0
Variable_34
Assign_34:0
Variable_35
Assign_35:0
Variable_36
Assign_36:0
Variable_37
Assign_37:0
Variable_38
Assign_38:0
Variable_39
Assign_39:0


In [8]:
para_dict={}
for k in tf.global_variables():
    if k not in tf.contrib.framework.get_variables_by_suffix('Momentum'):
            para_dict[k.name[:-2]] = k

In [9]:
para_toload={}
for k in tf.global_variables():
    if k not in tf.contrib.framework.get_variables_by_suffix('Momentum'):
        if 'mask' not in k.name:            
            para_toload[k.name[:-2]] = k

In [10]:
A_assign=[]
for k in tf.get_default_graph().get_operations():
    if k.name.startswith('Assign'):
        A_assign.append(k.values()[0])
update=tf.group(*A_assign)

In [11]:
sess=tf.InteractiveSession()
saver = tf.train.Saver(para_dict)
saver.restore(sess,'C:/Users/lhlne/Desktop\project/densenet/dnsmodel/9287adj/dns222_69.ckpt')

INFO:tensorflow:Restoring parameters from C:/Users/lhlne/Desktop\project/densenet/dnsmodel/9287adj/dns222_69.ckpt


In [12]:
para_dict

{'BatchNorm/beta': <tf.Variable 'BatchNorm/beta:0' shape=(16,) dtype=float32_ref>,
 'BatchNorm/gamma': <tf.Variable 'BatchNorm/gamma:0' shape=(16,) dtype=float32_ref>,
 'BatchNorm/moving_mean': <tf.Variable 'BatchNorm/moving_mean:0' shape=(16,) dtype=float32_ref>,
 'BatchNorm/moving_variance': <tf.Variable 'BatchNorm/moving_variance:0' shape=(16,) dtype=float32_ref>,
 'BatchNorm_1/beta': <tf.Variable 'BatchNorm_1/beta:0' shape=(28,) dtype=float32_ref>,
 'BatchNorm_1/gamma': <tf.Variable 'BatchNorm_1/gamma:0' shape=(28,) dtype=float32_ref>,
 'BatchNorm_1/moving_mean': <tf.Variable 'BatchNorm_1/moving_mean:0' shape=(28,) dtype=float32_ref>,
 'BatchNorm_1/moving_variance': <tf.Variable 'BatchNorm_1/moving_variance:0' shape=(28,) dtype=float32_ref>,
 'BatchNorm_10/beta': <tf.Variable 'BatchNorm_10/beta:0' shape=(136,) dtype=float32_ref>,
 'BatchNorm_10/gamma': <tf.Variable 'BatchNorm_10/gamma:0' shape=(136,) dtype=float32_ref>,
 'BatchNorm_10/moving_mean': <tf.Variable 'BatchNorm_10/moving

In [13]:
zero_list = dict()
for k in para_dict.items():
    if 'mask' in k[0]:
        a=np.sum(k[1].eval())
        b=np.shape(np.reshape(k[1].eval(),[-1]))
        zero_list[k[0].rstrip('mask')] = a/b
        print (k[0])

Variablemask
Variable_1mask
Variable_2mask
Variable_3mask
Variable_4mask
Variable_5mask
Variable_6mask
Variable_7mask
Variable_8mask
Variable_9mask
Variable_10mask
Variable_11mask
Variable_12mask
Variable_13mask
Variable_14mask
Variable_15mask
Variable_16mask
Variable_17mask
Variable_18mask
Variable_19mask
Variable_20mask
Variable_21mask
Variable_22mask
Variable_23mask
Variable_24mask
Variable_25mask
Variable_26mask
Variable_27mask
Variable_28mask
Variable_29mask
Variable_30mask
Variable_31mask
Variable_32mask
Variable_33mask
Variable_34mask
Variable_35mask
Variable_36mask
Variable_37mask
Variable_38mask
Variable_39mask


In [14]:
zero_list

{'Variable': array([ 0.75]),
 'Variable_1': array([ 0.31134259]),
 'Variable_10': array([ 0.17152031]),
 'Variable_11': array([ 0.28989651]),
 'Variable_12': array([ 0.21709209]),
 'Variable_13': array([ 0.59417969]),
 'Variable_14': array([ 0.23935185]),
 'Variable_15': array([ 0.28504522]),
 'Variable_16': array([ 0.27194042]),
 'Variable_17': array([ 0.27428193]),
 'Variable_18': array([ 0.26402244]),
 'Variable_19': array([ 0.25025253]),
 'Variable_2': array([ 0.04596561]),
 'Variable_20': array([ 0.28037197]),
 'Variable_21': array([ 0.25842441]),
 'Variable_22': array([ 0.29575376]),
 'Variable_23': array([ 0.2962963]),
 'Variable_24': array([ 0.29828042]),
 'Variable_25': array([ 0.29623288]),
 'Variable_26': array([ 0.62132098]),
 'Variable_27': array([ 0.34353679]),
 'Variable_28': array([ 0.33787506]),
 'Variable_29': array([ 0.34437105]),
 'Variable_3': array([ 0.0900463]),
 'Variable_30': array([ 0.34417211]),
 'Variable_31': array([ 0.33730535]),
 'Variable_32': array([ 0.

In [24]:
avg1 =0
avg2 =0
avg3 = 0
for k in zero_list.items():
    if k[0] in config.block_1: avg1 += k[1]
    if k[0] in config.block_2: avg2 += k[1]
    if k[0] in config.block_3: avg3 += k[1]

In [29]:
1-avg1/len(config.block_1)


array([ 0.82251511])

In [30]:
1-avg2/len(config.block_2)


array([ 0.72414549])

In [31]:
1-avg3/len(config.block_3)

array([ 0.79007809])

In [ ]:
1-avg3/len(config.block_3)

In [10]:
import config


In [11]:
def apply_prune(weights, prune_dict):
    for target in config.exc_para:
        wl = target

        # Get target layer's weights
        weight_obj = weights[wl]
        weight_arr = weight_obj.eval()
        weight_rest = weight_arr[prune_dict[wl] != 0]
        dic_tem = prune_dict[wl][prune_dict[wl] != 0]
        idx_rest = np.argsort(abs(np.reshape(weight_rest,[-1])))
        if config.prune_para[wl] == 'all':
            num_prune = int(sum(np.reshape(prune_dict[wl],-1)))
        else:    
            num_prune = int(len(np.reshape(weight_arr,[-1]))*config.prune_para[wl])
        print(sum(np.reshape(prune_dict[wl],-1)))
        print(np.shape(np.reshape(prune_dict[wl],-1)))
        weight_toprune = weight_rest[idx_rest[:num_prune]] 
        
        weight_toprune = np.zeros_like(weight_toprune)

        
        weight_rest[idx_rest[:num_prune]] = weight_toprune
        weight_arr[prune_dict[wl] != 0] =  weight_rest
        dic_tem [idx_rest[:num_prune]] =  np.zeros_like(dic_tem [idx_rest[:num_prune]])
        prune_dict[wl][prune_dict[wl] != 0] = dic_tem
        # Apply pruning
        print(sum(np.reshape(prune_dict[wl],-1)))

        # Store pruned weights as tensorflow objects
        sess.run(weight_obj.assign(weight_arr))

    return prune_dict

In [12]:
prune_dict = {}
for target in config.all_para:
    wl =target
    weight_obj = para_dict[wl]
    prune_dict[wl] = np.ones_like(weight_obj.eval())

In [14]:
saver.save(sess,'./modellog/densenetprune50.ckpt')

'./modellog/densenetprune50.ckpt'

In [22]:
import pickle
# create dict
# save dict
f1 = open("C:/Users/lhlne/Desktop/project/deep-models-master/densenet/prunemodel/stage1/prune50.txt","wb")
pickle.dump(prune_dict, f1)
f1.close()
# load dict
f2 = open("C:/Users/lhlne/Desktop/project/deep-models-master/densenet/prunemodel/stage1/prune50.txt","rb")
load_list = pickle.load(f2)
f2.close()
# print 
print(load_list)

{'Variable': array([[[[ 1.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,
           0.,  0.,  0.],
         [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  1.],
         [ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,
           0.,  1.,  0.]],

        [[ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,
           0.,  0.,  1.],
         [ 0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,
           0.,  0.,  1.],
         [ 1.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,
           0.,  0.,  1.]],

        [[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,
           0.,  0.,  0.],
         [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,
           0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,
           0.,  0.,  0.]]],


       [[[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,  0., 

In [16]:
from matplotlib import pyplot as plt
for i in para_dict.items():
    if 'Variable' in i[0]:
        a = para_dict[i[0]].eval()
        b = np.reshape(a,[-1])
        c = sum(b ==0)
        d = np.shape(b)
   #     e = np.percentile(b, 75)
        print(c)
        print(d)
 #       print(e)
   #     print(int(np.log10(abs(e))))
        print('percentage',c/d)
#         print(sum(b ==0))
#         plt.hist(abs(b)) 
#         plt.title(i[0]) 
#         plt.show()

216
(432,)
percentage [ 0.5]
864
(1728,)
percentage [ 0.5]
1512
(3024,)
percentage [ 0.5]
2160
(4320,)
percentage [ 0.5]
2808
(5616,)
percentage [ 0.5]
3456
(6912,)
percentage [ 0.5]
4104
(8208,)
percentage [ 0.5]
4752
(9504,)
percentage [ 0.5]
5400
(10800,)
percentage [ 0.5]
6048
(12096,)
percentage [ 0.5]
6696
(13392,)
percentage [ 0.5]
7344
(14688,)
percentage [ 0.5]
7992
(15984,)
percentage [ 0.5]
12800
(25600,)
percentage [ 0.5]
8640
(17280,)
percentage [ 0.5]
9288
(18576,)
percentage [ 0.5]
9936
(19872,)
percentage [ 0.5]
10584
(21168,)
percentage [ 0.5]
11232
(22464,)
percentage [ 0.5]
11880
(23760,)
percentage [ 0.5]
12528
(25056,)
percentage [ 0.5]
13176
(26352,)
percentage [ 0.5]
13824
(27648,)
percentage [ 0.5]
14472
(28944,)
percentage [ 0.5]
15120
(30240,)
percentage [ 0.5]
15768
(31536,)
percentage [ 0.5]
46208
(92416,)
percentage [ 0.5]
16416
(32832,)
percentage [ 0.5]
17064
(34128,)
percentage [ 0.5]
17712
(35424,)
percentage [ 0.5]
18360
(36720,)
percentage [ 0.5]
1900

In [45]:
a = parm_dict['Variable_39'].eval()
b = np.reshape(a,[-1])
print(sum(b <=0.00001))
print(np.shape(b))
print('percentage',a/b)

(4480,)

In [27]:
parm_dict={}
for k in tf.trainable_variables():
    parm_dict[k.name] = k
with tf.Session(graph=graph) as session:
    saver= tf.train.Saver(parm_dict) 
    saver.restore(session,"weights_densenet")

Variable:0
BatchNorm/beta:0
BatchNorm/gamma:0
Variable_1:0
BatchNorm_1/beta:0
BatchNorm_1/gamma:0
Variable_2:0
BatchNorm_2/beta:0
BatchNorm_2/gamma:0
Variable_3:0
BatchNorm_3/beta:0
BatchNorm_3/gamma:0
Variable_4:0
BatchNorm_4/beta:0
BatchNorm_4/gamma:0
Variable_5:0
BatchNorm_5/beta:0
BatchNorm_5/gamma:0
Variable_6:0
BatchNorm_6/beta:0
BatchNorm_6/gamma:0
Variable_7:0
BatchNorm_7/beta:0
BatchNorm_7/gamma:0
Variable_8:0
BatchNorm_8/beta:0
BatchNorm_8/gamma:0
Variable_9:0
BatchNorm_9/beta:0
BatchNorm_9/gamma:0
Variable_10:0
BatchNorm_10/beta:0
BatchNorm_10/gamma:0
Variable_11:0
BatchNorm_11/beta:0
BatchNorm_11/gamma:0
Variable_12:0
BatchNorm_12/beta:0
BatchNorm_12/gamma:0
Variable_13:0
BatchNorm_13/beta:0
BatchNorm_13/gamma:0
Variable_14:0
BatchNorm_14/beta:0
BatchNorm_14/gamma:0
Variable_15:0
BatchNorm_15/beta:0
BatchNorm_15/gamma:0
Variable_16:0
BatchNorm_16/beta:0
BatchNorm_16/gamma:0
Variable_17:0
BatchNorm_17/beta:0
BatchNorm_17/gamma:0
Variable_18:0
BatchNorm_18/beta:0
BatchNorm_18

In [7]:
depth = 40
weight_decay = 1e-4
layers = int((depth - 4) / 3)

xs = tf.placeholder("float", shape=[None, image_dim])
ys = tf.placeholder("float", shape=[None, label_count])
lr = tf.placeholder("float", shape=[])
keep_prob = tf.placeholder(tf.float32)
is_training = tf.placeholder("bool", shape=[])


current = tf.reshape(xs, [ -1, 32, 32, 3 ])
current = conv2d(current, 3, 16, 3)

current, features = block(current, layers, 16, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)

current = tf.contrib.layers.batch_norm(current, scale=True, is_training=is_training, updates_collections=None)
current = tf.nn.relu(current)
current = avg_pool(current, 8)
final_dim = features
current = tf.reshape(current, [ -1, final_dim ])
Wfc = weight_variable([ final_dim, label_count ])
bfc = bias_variable([ label_count ])
ys_ = tf.nn.softmax( tf.matmul(current, Wfc) + bfc )

cross_entropy = -tf.reduce_mean(ys * tf.log(ys_ + 1e-12))
l2 = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
train_step = tf.train.MomentumOptimizer(lr, 0.9, use_nesterov=True).minimize(cross_entropy + l2 * weight_decay)
correct_prediction = tf.equal(tf.argmax(ys_, 1), tf.argmax(ys, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.Session()
batch_size = 64
learning_rate = 0.1
tf.global_variables_initializer()
    # saver = tf.train.Saver()
    # train_data, train_labels = data['train_data'], data['train_labels']
    # batch_count = int(len(train_data) / batch_size)
    # batches_data = np.split(train_data[:batch_count * batch_size], batch_count)
    # batches_labels = np.split(train_labels[:batch_count * batch_size], batch_count)
    # print ("Batch per epoch: ", batch_count)
    # for epoch in range(1, 1+300):
    #   if epoch == 150: learning_rate = 0.01
    #   if epoch == 225: learning_rate = 0.001
    #   for batch_idx in range(batch_count):
    #     xs_, ys_ = batches_data[batch_idx], batches_labels[batch_idx]
    #     batch_res = session.run([ train_step, cross_entropy, accuracy ],
    #       feed_dict = { xs: xs_, ys: ys_, lr: learning_rate, is_training: True, keep_prob: 0.8 })
    #     if batch_idx % 100 == 0: print (epoch, batch_idx, batch_res[1:])

    #   save_path = saver.save(session, 'densenet_%d.ckpt' % epoch)
    #   test_results = run_in_batch_avg(session, [ cross_entropy, accuracy ], [ xs, ys ],
    #       feed_dict = { xs: data['test_data'], ys: data['test_labels'], is_training: False, keep_prob: 1. })
    #   print (epoch, batch_res[1:], test_results)


<tf.Operation 'init' type=NoOp>